In [1]:
import cv2
import numpy
import numpy as np
import serial
import math
import matplotlib.pyplot as plt
import serial
import time

In [2]:
# 打开摄像机，不轻易运行
capture=cv2.VideoCapture(1)

In [4]:
# 定义何时的卷积核
max_speed=6
def convert_to_two_digits(number):
    if 10 > number >= 0:
        return '+0' + str(number)
    if 100 > number >= 10:
        return '+' + str(min(number,max_speed))
    if 0 > number > -10:
        return '-0' + str(number)[1]
    if -10 >= number > -100:
        return '-'+str(min(abs(number),max_speed))
# ser.close()
ser = serial.Serial('COM10', 115200)

In [5]:

 # 以下是静态工作区-----------------------------------------------------------------------------------
a=17
b=16
#首先，获取最主要的路径
_,image1=capture.read()
# 先转为灰度图
gray=cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
# 先获取边界
edges=cv2.Canny(gray, 240, 200)
#二极化
# _, binary = cv2.threshold(edges, 60, 255, cv2.THRESH_BINARY)
# 获取边界当中的主要路径
# road_indistinct = cv2.filter2D(binary, -2, kernel)
#二极化获取最终的路径
# _, road = cv2.threshold(road_indistinct, 100, 255, cv2.THRESH_BINARY)
'''
#获取路径上每个点的坐标
circle= np.array(np.where(edges)).T

# 获得边框中心点的坐标
circle_center=np.ones(2)
circle_center[0]=int(np.sum(circle[:,0])/(np.size(circle[:,0])+0.001))
circle_center[1]=int(np.sum(circle[:,1])/(np.size(circle[:,1])+0.001))

# 计算每个点与中心点的距离
dis=circle-circle_center
dis=np.linalg.norm(dis,axis=1)

# 获取长宽
road_width=min(dis)*2
road_length=np.sqrt(max(dis)**2-min(dis)**2)*2
# 获取长方形四个边界点的位置
contours, _ = cv2.findContours(road, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# '''
# plt.imshow(road, cmap='gray')

'\n#获取路径上每个点的坐标\ncircle= np.array(np.where(edges)).T\n\n# 获得边框中心点的坐标\ncircle_center=np.ones(2)\ncircle_center[0]=int(np.sum(circle[:,0])/(np.size(circle[:,0])+0.001))\ncircle_center[1]=int(np.sum(circle[:,1])/(np.size(circle[:,1])+0.001))\n\n# 计算每个点与中心点的距离\ndis=circle-circle_center\ndis=np.linalg.norm(dis,axis=1)\n\n# 获取长宽\nroad_width=min(dis)*2\nroad_length=np.sqrt(max(dis)**2-min(dis)**2)*2\n# 获取长方形四个边界点的位置\ncontours, _ = cv2.findContours(road, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)\n# '

In [6]:
# data = "ffffff"
# encoded_data = data.encode()
# ser.write(encoded_data)
# ser.close()
# ser = serial.Serial('COM10', 115200)
# # 第1个数正上负下
rotate=np.ones(2)
rotate[0]=6
# 第2个数，正左负右
rotate[1]=6
data = convert_to_two_digits(int(rotate[0]))+convert_to_two_digits(int(rotate[1]))
encoded_data = data.encode()
ser.write(encoded_data)

6

In [ ]:
''' 以下是动态工作区-----------------------------------------------------------------------------------'''
dis_center=12
# 首先将中心点移到指定位置------------------------------------------------------------------------------
while True:
    _,image=capture.read()
    # 找到红色的点
    circle= np.array(np.where(edges))
    image_red_true=image[:,:,2]
    image_back=(image[:,:,1]+image[:,:,0])/2
    image_red=image_red_true-image_back
    red = np.array(np.argwhere(image_red>240))
    # 红色中心点坐标
    if red.shape[0]>2:
        red_center=np.ones(2)
        red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
        red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
    else:
        break
    image_green_true=image[:,:,1]
    image_back=(image[:,:,2]+image[:,:,0])/2
    image_green=image_red_true-image_back
    green = np.array(np.argwhere(image_green>240))
    # 绿中心点坐标
    if green.shape[0]>2:
        green_center=np.ones(2)
        green_center[0]=int(np.sum(green[:,0])/(np.size(green[:,0])+0.001))
        green_center[1]=int(np.sum(green[:,1])/(np.size(green[:,0])+0.001))
    else:
        break
    dis_center=np.linalg.norm(green_center-red_center)
    while dis_center>3:
        _,image=capture.read()
        # 找到红色的点
        circle= np.array(np.where(edges))
        image_red_true=image[:,:,2]
        image_back=(image[:,:,1]+image[:,:,0])/2
        image_red=image_red_true-image_back
        red = np.array(np.argwhere(image_red>240))
        # 红色中心点坐标
        if red.shape[0]>2:
            red_center=np.ones(2)
            red_center[0]=int(np.sum(red[:,0])/(np.size(red[:,0])+0.001))
            red_center[1]=int(np.sum(red[:,1])/(np.size(red[:,0])+0.001))
        else:
            break
        image_green_true=image[:,:,1]
        image_back=(image[:,:,2]+image[:,:,0])/2
        image_green=image_red_true-image_back
        green = np.array(np.argwhere(image_green>240))
        # 绿中心点坐标
        if green.shape[0]>2:
            green_center=np.ones(2)
            green_center[0]=int(np.sum(green[:,0])/(np.size(green[:,0])+0.001))
            green_center[1]=int(np.sum(green[:,1])/(np.size(green[:,0])+0.001))
        else:
            break
        dis_center=np.linalg.norm(green_center-red_center)
        rotate=(green_center-red_center)/3
        rotate=-rotate
        # 第1个数正上负下  第2个数，正左负右
        data = convert_to_two_digits(int(rotate[0]))+convert_to_two_digits(int(rotate[1]))
        encoded_data = data.encode()
        ser.write(encoded_data)
        time.sleep(0.1)
        # 找到四边形的四个边界点----------------------------------------------------------------------

In [6]:
# 第1个数正上负下
rotate=np.ones(2)
rotate[0]=-10
# 第2个数，正左负右
rotate[1]=-10
data = convert_to_two_digits(int(rotate[0]))+convert_to_two_digits(int(rotate[1]))
encoded_data = data.encode()
ser.write(encoded_data)

6